In [0]:
#https://blog.csdn.net/u013714645/article/details/97899342
import pandas as pd
from deepctr.inputs import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
from google.colab import drive
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import math
from sklearn.externals import joblib
import os
import random
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import csv

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! pip install deepmatch
! pip install deepctr
! pip install faiss-cpu

     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 7.2MB 13.4MB/s 


In [0]:
! wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231785/underexpose_train.zip -O /content/gdrive/My Drive/underexpose_train.zip 
! wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231785/underexpose_test.zip -O /content/gdrive/My Drive/underexpose_test.zip
! unzip -o /content/gdrive/My Drive/underexpose_train.zip  
! unzip -o /content/gdrive/My Drive/underexpose_test.zip 

In [0]:
!mv "underexpose_train/underexpose_train_click-0.csv" "/.." 

In [0]:
%cd /content/gdrive/My Drive/project_data

In [0]:
!ls

In [0]:
!mv 'underexpose_user_feat.csv' '/content'

In [0]:
!7z e -pdee22d5e4a7b1e3c409ea0719aa0a715 underexpose_test/underexpose_test_click-6.zip

In [0]:
def gen_data_set(data, negsample=0):

    data.sort_values("timestamp", inplace=True)
    item_ids = data['movie_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
  
        pos_list = hist['movie_id'].tolist()
        rating_list = hist['rating'].tolist()

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]),rating_list[i]))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else:
                test_set.append((reviewerID, hist[::-1], pos_list[i], 1,len(hist[::-1]),rating_list[i]))

    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "movie_id": train_iid, "hist_movie_id": train_seq_pad,
                         "hist_len": train_hist_len}

    for key in ["gender", "age", "occupation"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input, train_label

In [0]:
def read_item_feat(path):
    col_name = ['item_id']
    for i in range(256):
      col_name.append(str(i))
    
    item_feat = pd.read_csv(path, header=None, names=col_name)
    
    item_feat.iloc[:,   1] = [float(i) for i in item_feat.iloc[:,   1].str[1:]]
    item_feat.iloc[:, 128] = [float(i) for i in item_feat.iloc[:, 128].str[:-1]]
    item_feat.iloc[:, 129] = [float(i) for i in item_feat.iloc[:, 129].str[1:]]
    item_feat.iloc[:, 256] = [float(i) for i in item_feat.iloc[:, 256].str[:-1]]
    return item_feat

In [0]:
def get_item_cnt(df_, user_col, item_col):
    sim_txt, sim_img = joblib.load(os.path.join(data_path+'sim_dict.pkl'))

    df = df_.copy()
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))

    rateing=list()
    total = 0
    item_cnt = defaultdict(int)  # 商品被点击次数
    for user, items in tqdm(user_item_dict.items()):
        for loc1, item in enumerate(items):
            total += 1
            item_cnt[item] += 1   
            rate = 0
            if item in sim_txt.keys() and item in sim_img.keys():
                for ritem, sim in sim_txt[item].items():
                    if ritem in items and ritem !=item:
                        rate += sim
                for ritem, sim in sim_img[item].items():   
                    if ritem in items and ritem !=item:
                        rate += sim
                rate /= len(items*2)
            rateing.append([user, item, rate])
    for i, rlist in enumerate(rateing):
        if rate == 0:
            rateing[i] = [rlist[0], rlist[1], item_cnt[item]/total]
    return pd.DataFrame([(a, b, c) for a, b, c in rateing], columns=['user_id', 'movie_id', 'rating'])

In [34]:
data_path = '/content/gdrive/My Drive/project_data/'
now_phase = 6
#item_feat = read_item_feat(data_path+'underexpose_item_feat.csv')
user_feat = pd.read_csv(data_path+'underexpose_user_feat.csv', header=None, names=['user_id', 'age', 'gender', 'occupation'])
user_feat['gender'] = (user_feat['gender'] != 'M').astype(int)

test_click = pd.DataFrame()
train_click = pd.DataFrame()
for c in range(now_phase + 1):
    test_tmp = pd.read_csv(data_path + '/underexpose_test_click-{}.csv'.format(c), header=None, names=['user_id', 'movie_id', 'timestamp'])
    train_tmp = pd.read_csv(data_path + '/underexpose_train_click-{}.csv'.format(c), header=None, names=['user_id', 'movie_id', 'timestamp'])

    test_click = test_click.append(test_tmp)
    test_click = test_click.drop_duplicates(subset=['user_id', 'movie_id', 'timestamp'], keep='last')
    train_click = train_click.append(test_tmp)
    train_click = train_click.append(train_tmp)
    train_click = train_click.drop_duplicates(subset=['user_id', 'movie_id', 'timestamp'], keep='last')
    train_click = train_click.sort_values('timestamp')

click_to_rating = get_item_cnt(train_click, 'user_id', 'movie_id')

test_click = pd.merge(test_click, user_feat, how='left', on=['user_id'])
train_click = pd.merge(train_click, user_feat, how='left', on=['user_id'])

test_click = pd.merge(test_click, click_to_rating, how='left', on=['user_id','movie_id'])
train_click = pd.merge(train_click, click_to_rating, how='left', on=['user_id','movie_id'])

#test_click = pd.merge(test_click, item_feat, how='left', on=['item_id'])
#train_click = pd.merge(train_click, item_feat, how='left', on=['item_id'])
test_click = test_click.fillna(test_click.mean())
train_click = train_click.fillna(train_click.mean())

100%|██████████| 31525/31525 [01:49<00:00, 288.10it/s]


In [37]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
data = train_click.copy()

sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", ]
SEQ_LEN = 50
negsample = 0

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`
labelencoder_dict = {}
feature_max_idx = {}
features = ['user_id', 'movie_id', 'gender', 'age', 'occupation']
for feature in features:
    lbe = LabelEncoder()
    if feature == 'user_id' or feature == 'movie_id':
        tmp_data = data[feature].copy()
        data[feature] = lbe.fit_transform(data[feature])
        feature_max_idx[feature] = data[feature].max() 
        labelencoder_dict[feature]=lbe
        #　print(labelencoder_dict[feature].inverse_transform(data.head()[feature]))
        
    else:
        data[feature] = lbe.fit_transform(data[feature]) +1
        feature_max_idx[feature] = data[feature].max() +1

user_profile = data[["user_id", "gender", "age", "occupation"]].drop_duplicates('user_id')

item_profile = data[["movie_id"]].drop_duplicates('movie_id')


user_profile.set_index("user_id", inplace=True)

user_item_list = data.groupby("user_id")['movie_id'].apply(list)

train_set, test_set= gen_data_set(data, negsample)

train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

# 2.count #unique features for each sparse field and generate feature config for sequence feature

embedding_dim = 32

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]

# 3.Define Model and train

K.set_learning_phase(True)

import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=100, user_dnn_hidden_units=(128,64, embedding_dim))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=False,p=1,k_max=2,num_sampled=100,user_dnn_hidden_units=(128,64, embedding_dim),init_std=0.001)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)


100%|██████████| 31525/31525 [00:14<00:00, 2246.73it/s]


6 6
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1057399 samples
Epoch 1/20
1057399/1057399 [==============================] - 243s 230us/sample - loss: 3.1099
Epoch 2/20
1057399/1057399 [==============================] - 239s 226us/sample - loss: 2.9326
Epoch 3/20
1057399/1057399 [==============================] - 241s 228us/sample - loss: 2.8797
Epoch 4/20
1057399/1057399 [==============================] - 239s 226us/sample - loss: 2.8336
Epoch 5/20
1057399/1057399 [==============================] - 240s 227us/sample - loss: 2.7580
Epoch 6/20
1057399/1057399 [==============================] - 240s 227us/sample - loss: 2.6534
Epoch 7/20
1057399/1057399 [==============================] - 242s 229us/sample - loss: 2.5641
Epoch 8/20
1057399/1057399 [==============================] - 240s 227us/sample - loss: 2.4860
Epoch 9/20
1

In [38]:
test_true_label = {line[0]:[line[2]] for line in test_set}
result = pd.DataFrame()
import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N
result = {}
index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
        result.setdefault(uid, list())  
        result[uid] = pred
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))
joblib.dump(result, os.path.join(data_path+'result.pkl'))

31525it [00:07, 4019.30it/s]



recall 0.023409992069785883
hit rate 0.023409992069785883


['/content/gdrive/My Drive/project_data/result.pkl']

In [0]:
with open(data_path+'underexpose_submit-{}.csv'.format(now_phase), 'w', newline='') as csvfile:
  writer  = csv.writer(csvfile)
  for k, v in result.items():
      row = labelencoder_dict['user_id'].inverse_transform([k])
      row = np.append(row, labelencoder_dict['movie_id'].inverse_transform(v), axis=0)
      writer.writerow(row)

In [0]:
'''
col_name = ['user_id']
for i in range(50):
  col_name.append(str(i))
a = pd.read_csv(data_path+'underexpose_submit-{}.csv'.format(now_phase), header=None, names=col_name)
b = pd.DataFrame()
for c in range(now_phase + 1):
    test_tmp = pd.read_csv(data_path + 'underexpose_test_qtime-{}.csv'.format(c), header=None, names=['user_id', 'timestamp'])
    b = b.append(test_tmp)
    b = b.drop_duplicates(subset=['user_id', 'timestamp'], keep='last')

print(a.head())
print(b.head())
b = pd.merge(b, a, how='left', on=['user_id'])
b.drop('timestamp', axis=1, inplace=True)  
b = b.drop_duplicates(subset=['user_id'], keep='last')
print(b.head())
b.to_csv(data_path + 'underexpose_submit-T.csv', index=False, header=None)
'''